# Homework 4

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2025

***

### Data

The file `data/ratetree_data_2025-01-31.xlsx` has a binomial tree of interest rates fit to...
* discount curves from `cap_curves_2025-01-31.xlsx`
* implied vols from `cap_curves_2025-01-31.xlsx`

Note the following...
* Suppose the present date is `2025-01-31`.
* The rates are continuously compounded.
* The rates are for the following quarter. So teh rate at $t=0$ is the continuously compounded rate for the interval $t=0$ to $t=.25$.

Take this binomial tree as given; there is no need to fit it yourself.

In [90]:
import pandas as pd

DATE = '2025-01-31'
FILEIN = f'../data/ratetree_data_{DATE}.xlsx'
sheet_tree = 'rate tree'

ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
ratetree.columns.name = 'time'

ratetree.style.format('{:.1%}',na_rep='').format_index('{:.2f}',axis=1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,4.2%,4.3%,4.4%,4.8%,5.1%,6.4%,7.6%,9.1%,10.1%,11.8%,13.5%,15.3%,16.9%,19.2%,22.7%,25.9%,28.2%,30.6%,34.7%,40.3%
1,,3.9%,4.0%,4.2%,4.4%,5.2%,6.0%,7.0%,7.8%,9.0%,10.4%,11.8%,13.0%,14.8%,17.4%,19.9%,21.7%,23.7%,27.0%,31.2%
2,,,3.6%,3.7%,3.8%,4.2%,4.7%,5.4%,6.0%,7.0%,8.0%,9.1%,10.1%,11.4%,13.4%,15.3%,16.7%,18.4%,20.9%,24.2%
3,,,,3.3%,3.2%,3.3%,3.7%,4.2%,4.6%,5.3%,6.2%,7.0%,7.8%,8.8%,10.3%,11.8%,12.9%,14.2%,16.3%,18.8%
4,,,,,2.8%,2.7%,2.9%,3.3%,3.6%,4.1%,4.7%,5.4%,6.0%,6.8%,7.9%,9.1%,10.0%,11.0%,12.6%,14.6%
5,,,,,,2.2%,2.3%,2.5%,2.7%,3.2%,3.6%,4.2%,4.6%,5.2%,6.1%,7.0%,7.7%,8.6%,9.8%,11.3%
6,,,,,,,1.8%,2.0%,2.1%,2.4%,2.8%,3.2%,3.6%,4.0%,4.7%,5.4%,5.9%,6.6%,7.6%,8.8%
7,,,,,,,,1.5%,1.6%,1.9%,2.2%,2.5%,2.7%,3.1%,3.6%,4.1%,4.6%,5.1%,5.9%,6.8%
8,,,,,,,,,1.3%,1.4%,1.7%,1.9%,2.1%,2.4%,2.8%,3.2%,3.5%,4.0%,4.6%,5.3%


***

In [91]:
capcurves = pd.read_excel("../data/cap_curves_2025-01-31.xlsx")
capcurves

,tenor,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
0,0.25,0.042340,0.042340,0.989526,NaN,NaN,NaN
1,0.50,0.041854,0.041852,0.979398,0.041364,0.101500,0.101500
2,0.75,0.041404,0.041397,0.969584,0.040489,0.116946,0.128478
3,1.00,0.041029,0.041018,0.960012,0.039882,0.132391,0.154562
4,1.25,0.040458,0.040438,0.950950,0.038117,0.159421,0.219138
5,1.50,0.040142,0.040117,0.941881,0.038514,0.180856,0.239613
6,1.75,0.039966,0.039940,0.932816,0.038873,0.197446,0.254106
7,2.00,0.039902,0.039877,0.923708,0.039440,0.209941,0.261605
8,2.25,0.039717,0.039688,0.914976,0.038173,0.219090,0.263323
9,2.50,0.039636,0.039606,0.906171,0.038868,0.225643,0.261958


# 1. Binomial Tree Pricing - Bond

### The Bond

Consider a vanilla (non-callable) bond with the following parameters...
* `T=5`
* coupon rate is `4.41%`
* coupons are semiannual

Note that this is essentially the hypothetical bond priced in HW 1.

### 1.1

Create and display a tree of cashflows from the bond, corresponding to each node of the tree (state and time) seen in the interest rate tree.

Note that the cashflows do not depend on the interest rates. Thus, report the cashflows at the time (in the column) they are actually paid out. The final payoff (face plus coupon) occurs at $T$, which is beyond the interest rate tree. You are welcome to add a column for $T$ or to consider this payoff separately and leave it out of the tree.

In [92]:
face = 100
coupon_rate = 0.0441
coupon = face * coupon_rate / 2

times = ratetree.columns.astype(float)
cf_bond = pd.DataFrame(0, index=ratetree.index, columns=ratetree.columns)

for t in times:
    if (t > 0) and ((t * 2).is_integer()) and (t <= 5):
        cf_bond[t] = coupon
    if t == 5:
        cf_bond[t] += face

cf_bond

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5,4.75
state,,,,,,,,,,,,,,,,,,,,
0,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0
1,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0
2,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0
3,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0
4,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0
5,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0
6,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0
7,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0
8,0,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0,2.205,0


### 1.2.

Create and display a tree of values of the bond. Do this for the quotes as
* clean quotes
* dirty quotes

Given the semiannual coupons and quarterly tree steps, the clean and dirty will coincide at $t=0, .5, 1,...$.

Do the valuation by...
* setting the value at $T$ as the face plus final coupon.
* discounting this back through time, using the (continuously-compounded) interest rate.
* recall that the tree is constructed such that the probability of moving "up" or "down" is 50%.

In [93]:
import numpy as np

face = 100
coupon_rate = 0.0441
coupon = face * coupon_rate / 2

# Make sure cf_bond is float from the start
cf_bond = cf_bond.astype(float)

# Add a column at T=5 to ratetree (approximate rates) and cf_bond (final payoff)
if 5.0 not in ratetree.columns:
    ratetree[5.0] = ratetree.iloc[:, -1]
    ratetree = ratetree.reindex(sorted(ratetree.columns), axis=1)
if 5.0 not in cf_bond.columns:
    cf_bond[5.0] = face + coupon

val_dirty = cf_bond.copy()  # already float
times = sorted(ratetree.columns)

# Backward recursion for dirty price
for i in range(len(times) - 2, -1, -1):
    t = times[i]
    t_next = times[i + 1]
    dt = t_next - t
    for s in val_dirty.index:
        r = ratetree.loc[s, t]
        up = s + 1 if (s + 1 in val_dirty.index) else s
        val_dirty.loc[s, t] += 0.5 * np.exp(-r * dt) * (
            val_dirty.loc[s, t_next] + val_dirty.loc[up, t_next]
        )

def accrued_interest(t):
    last_coupon = np.floor(t * 2) / 2
    fraction = (t - last_coupon) / 0.5
    return coupon * max(fraction, 0)

val_clean = val_dirty.copy()
for t in val_clean.columns:
    val_clean[t] = val_dirty[t] - accrued_interest(float(t))


In [94]:
val_dirty

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
state,,,,,,,,,,,,,,,,,,,,,
0,101.869401,101.020606,100.038066,96.674771,95.412415,91.804347,90.561114,87.209059,86.276634,83.234494,...,80.182270,80.271327,78.422727,79.356835,78.871143,81.619125,83.017188,87.901687,92.411267,102.205
1,NaN,104.874751,104.198996,101.179152,100.262969,97.006495,96.027611,92.901158,92.134736,89.229230,...,86.297496,86.364216,84.453333,85.218808,84.423561,86.664663,87.393012,91.350206,94.526329,102.205
2,NaN,NaN,107.609029,104.868428,104.234514,101.267177,100.510592,97.577717,96.958541,94.177536,...,91.361871,91.411396,89.445063,90.062346,88.994402,90.795758,90.948229,94.121954,96.199500,102.205
3,NaN,NaN,NaN,107.861708,107.450967,104.713944,104.138429,101.366170,100.872595,98.200304,...,95.490134,95.526543,93.512637,94.004142,92.704542,94.136113,93.806834,96.332291,97.517148,102.205
4,NaN,NaN,NaN,NaN,110.034062,107.475596,107.043995,104.401717,104.012321,101.432140,...,98.814212,98.840717,96.787106,97.174574,95.683329,96.811009,96.086825,98.084197,98.551194,102.205
5,NaN,NaN,NaN,NaN,NaN,109.671862,109.352437,106.813373,106.508629,104.004803,...,101.465365,101.484465,99.398401,99.701469,98.054765,98.936965,97.893949,99.466184,99.360480,102.205
6,NaN,NaN,NaN,NaN,NaN,NaN,111.175060,108.716765,108.479758,106.038212,...,103.564265,103.577880,101.465706,101.701306,99.930349,100.616810,99.319321,100.552357,99.992531,102.205
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.211354,110.027926,107.636569,...,105.216479,105.226070,103.093144,103.275412,101.406252,101.938170,100.439347,101.403601,100.485356,102.205
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.238893,108.887614,...,106.511335,106.518000,104.368733,104.509208,102.563094,102.973917,101.316862,102.069250,100.869139,102.205


In [95]:
val_clean

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
state,,,,,,,,,,,,,,,,,,,,,
0,101.869401,99.918106,100.038066,95.572271,95.412415,90.701847,90.561114,86.106559,86.276634,82.131994,...,79.079770,80.271327,77.320227,79.356835,77.768643,81.619125,81.914688,87.901687,91.308767,102.205
1,NaN,103.772251,104.198996,100.076652,100.262969,95.903995,96.027611,91.798658,92.134736,88.126730,...,85.194996,86.364216,83.350833,85.218808,83.321061,86.664663,86.290512,91.350206,93.423829,102.205
2,NaN,NaN,107.609029,103.765928,104.234514,100.164677,100.510592,96.475217,96.958541,93.075036,...,90.259371,91.411396,88.342563,90.062346,87.891902,90.795758,89.845729,94.121954,95.097000,102.205
3,NaN,NaN,NaN,106.759208,107.450967,103.611444,104.138429,100.263670,100.872595,97.097804,...,94.387634,95.526543,92.410137,94.004142,91.602042,94.136113,92.704334,96.332291,96.414648,102.205
4,NaN,NaN,NaN,NaN,110.034062,106.373096,107.043995,103.299217,104.012321,100.329640,...,97.711712,98.840717,95.684606,97.174574,94.580829,96.811009,94.984325,98.084197,97.448694,102.205
5,NaN,NaN,NaN,NaN,NaN,108.569362,109.352437,105.710873,106.508629,102.902303,...,100.362865,101.484465,98.295901,99.701469,96.952265,98.936965,96.791449,99.466184,98.257980,102.205
6,NaN,NaN,NaN,NaN,NaN,NaN,111.175060,107.614265,108.479758,104.935712,...,102.461765,103.577880,100.363206,101.701306,98.827849,100.616810,98.216821,100.552357,98.890031,102.205
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.108854,110.027926,106.534069,...,104.113979,105.226070,101.990644,103.275412,100.303752,101.938170,99.336847,101.403601,99.382856,102.205
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.238893,107.785114,...,105.408835,106.518000,103.266233,104.509208,101.460594,102.973917,100.214362,102.069250,99.766639,102.205


### 1.3.

The binomial-estimated price of the bond is the initial node of the value tree.

Report this along with the price of the bond you would get from the usual simple formula for such a bond. 
* Consider pricing it with the $T$ interval swap rate (used similar to a ytm) from the file `cap_curves_2025-01-31.xlsx`.
* If you do this, recall that the swap rate given in that file is quarterly-compounded, so you would need to convert it to semiannual compounding before plugging it into the usual closed-form ytm-pricing formula.

In [ ]:
# Binomial price at the root node (t=0, state=0)
binomial_dirty = val_dirty.loc[0, 0]
binomial_clean = val_clean.loc[0, 0]

# 5-year swap rate (quarterly-compounded) from capcurves
swap_rate_5 = capcurves.loc[capcurves['tenor'] == 5, 'swap rates'].iloc[0]

# Convert quarterly-compounded swap rate to a semiannual yield
semiannual_yield = (1 + swap_rate_5 / 4) ** 2 - 1

# Closed-form price for semiannual coupon bond
n = 10  # 5 years * 2 coupons per year
price_closed_form = sum([coupon / (1 + semiannual_yield) ** k for k in range(1, n + 1)]) \
                    + face / (1 + semiannual_yield) ** n

binomial_dirty, binomial_clean, price_closed_form

(101.8694005797055, 101.8694005797055, 101.86976852197122)

### Note:

An easy check on your code is whether it will correctly price a zero-coupon bond at a price that matches the "discounts" in the `cap_curves` data file.

***

# 2. Pricing the Callable - European

### 2.1.

Calculate and display value tree of a European-style call option on the bond analyzed in part `1`.
* `$T_o = 3$`. That is, the time-to-expiration is 3 years.
* `$K=100$`. That is, the strike is 100. This is a clean strike, meaning exercise requires paying the strike plus any accrued interest.

Do so by 
* setting the value at the time of expiration, using the value of the bond for each node at that time.
* discounting this back through time, using the (continuously-compounded) interest rate.
* recall that the tree is constructed such that the probability of moving "up" or "down" is 50%.

Note that...
* the tree of call values will not be the same size as the tree of bond values. The former goes only to $T_o=3$.

In [98]:
T_o = 3.0
strike = 100
call_cols = sorted([t for t in ratetree.columns if float(t) <= T_o])
call_tree = pd.DataFrame(0.0, index=ratetree.index, columns=call_cols)

for s in call_tree.index:
    call_tree.loc[s, T_o] = max(val_clean.loc[s, T_o] - strike, 0)

for i in range(len(call_cols) - 2, -1, -1):
    t = call_cols[i]
    t_next = call_cols[i + 1]
    dt = t_next - t
    for s in call_tree.index:
        r = ratetree.loc[s, t]
        s_up = s + 1 if (s + 1 in call_tree.index) else s
        call_tree.loc[s, t] = np.exp(-r * dt) * 0.5 * (
            call_tree.loc[s, t_next] + call_tree.loc[s_up, t_next]
        )

call_tree

,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,3.118997,2.493648,1.881670,1.312683,0.820569,0.435626,0.177591,0.042461,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,3.810374,3.159831,2.492736,1.836379,1.226605,0.707802,0.319550,0.086867,0.000000,0.000000,0.000000,0.000000
2,NaN,NaN,4.535708,3.890733,3.201795,2.486561,1.777338,1.117432,0.563553,0.177161,0.000000,0.000000,0.000000
3,NaN,NaN,NaN,5.263394,4.651966,3.977347,3.247710,2.479418,1.701952,0.967019,0.360534,0.000000,0.000000
4,NaN,NaN,NaN,NaN,5.960785,5.401621,4.773626,4.076571,3.309536,2.476510,1.599512,0.732241,0.000000
5,NaN,NaN,NaN,NaN,NaN,6.602280,6.102253,5.540670,4.910669,4.201802,3.404615,2.504861,1.484465
6,NaN,NaN,NaN,NaN,NaN,NaN,7.173571,6.734186,6.241302,5.687132,5.065545,4.366654,3.577880
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.677996,7.293606,6.861557,6.377885,5.835675,5.226070
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.121192,7.785069,7.409315,6.989063,6.518000


### 2.2.

Show the value tree of the callable bond by subtracting the call value tree from the (subset $t\le T_o$ of the) bond value tree (calculated in part `1`.) Do this for both
* clean
* dirty

In [100]:
bond_dirty_subset = val_dirty[call_cols]
bond_clean_subset = val_clean[call_cols]

callable_dirty = bond_dirty_subset - call_tree
callable_clean = bond_clean_subset - call_tree

In [101]:
callable_dirty

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,98.750404,98.526958,98.156396,95.362088,94.591846,91.368721,90.383523,87.166598,86.276634,83.234494,82.683237,80.182270,80.271327
1,NaN,101.064377,101.039166,98.686416,98.426590,95.779890,95.319809,92.581608,92.047869,89.229230,88.757972,86.297496,86.364216
2,NaN,NaN,103.073320,100.977696,101.032718,98.780616,98.733254,96.460285,96.394988,94.000375,93.782819,91.361871,91.411396
3,NaN,NaN,NaN,102.598313,102.799001,100.736597,100.890719,98.886752,99.170643,97.233284,97.514257,95.490134,95.526543
4,NaN,NaN,NaN,NaN,104.073277,102.073975,102.270369,100.325146,100.702784,98.955629,99.567358,98.081971,98.840717
5,NaN,NaN,NaN,NaN,NaN,103.069582,103.250184,101.272704,101.597960,99.803002,100.385979,98.960505,100.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,104.001489,101.982579,102.238456,100.351081,100.800904,99.197611,100.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.533358,102.734320,100.775012,101.121707,99.380803,100.000000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.117701,101.102545,101.369526,99.522272,100.000000


In [102]:
callable_clean

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,98.750404,97.424458,98.156396,94.259588,94.591846,90.266221,90.383523,86.064098,86.276634,82.131994,82.683237,79.079770,80.271327
1,NaN,99.961877,101.039166,97.583916,98.426590,94.677390,95.319809,91.479108,92.047869,88.126730,88.757972,85.194996,86.364216
2,NaN,NaN,103.073320,99.875196,101.032718,97.678116,98.733254,95.357785,96.394988,92.897875,93.782819,90.259371,91.411396
3,NaN,NaN,NaN,101.495813,102.799001,99.634097,100.890719,97.784252,99.170643,96.130784,97.514257,94.387634,95.526543
4,NaN,NaN,NaN,NaN,104.073277,100.971475,102.270369,99.222646,100.702784,97.853129,99.567358,96.979471,98.840717
5,NaN,NaN,NaN,NaN,NaN,101.967082,103.250184,100.170204,101.597960,98.700502,100.385979,97.858005,100.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,104.001489,100.880079,102.238456,99.248581,100.800904,98.095111,100.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.430858,102.734320,99.672512,101.121707,98.278303,100.000000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.117701,100.000045,101.369526,98.419772,100.000000


### 2.3.

Report the initial node value of the call option and of the callable bond.

In a table, compare these to what you got in HW 1 as the value of the embedded call and the value of the callable bond.
* In `HW 1`, we were valuing from a date nearly two weeks later, `2025-02-13`. This difference in the timing means we wouldn't expect the values to match exactly, even if the methods were entirely consistent.

In [106]:
initial_call = call_tree.loc[0, 0]
initial_callable_dirty = callable_dirty.loc[0, 0]
initial_callable_clean = callable_clean.loc[0, 0]

comparison = pd.DataFrame({
    "Dirty (2025-01-31)": [initial_call, initial_callable_dirty],
    "Clean (2025-01-31)": [initial_call, initial_callable_clean],
    "HW1 (2025-02-13)": ["2.90", "98.67"]
}, index=["Embedded Call", "Callable Bond"])

comparison


,Dirty (2025-01-31),Clean (2025-01-31),HW1 (2025-02-13)
Embedded Call,3.118997,3.118997,2.90
Callable Bond,98.750404,98.750404,98.67


**I pulled the values for HW1 from question 1.4, which showed the Bond/Call values calculated via Black's formula.**

***

# 3. Pricing the Callable - American

### 3.1.

Re-do part `2.`, but this time, make the option a **American** style. That is, allow it to be exercised at any node.
* Report the tree of callable-bond values.
* How does this compare to the European-style?

#### Note
To do this valuation, go through the procedure in `2.1.`, but at each node, compare the value for the call with the value of the payoff function based on the vanilla bond's value at that node. Take the maximum of the two. If you code this carefully, you can simply add a line of code to what you did in `2.1`.

In [108]:
# American-style call option tree on the bond
call_tree_am = pd.DataFrame(0.0, index=ratetree.index, columns=call_cols)

# At expiration T_o, option value = max(bond clean value - strike, 0)
for s in call_tree_am.index:
    call_tree_am.loc[s, T_o] = max(val_clean.loc[s, T_o] - strike, 0)

# Backward recursion with early exercise
for i in range(len(call_cols) - 2, -1, -1):
    t = call_cols[i]
    t_next = call_cols[i + 1]
    dt = t_next - t
    for s in call_tree_am.index:
        r = ratetree.loc[s, t]
        s_up = s + 1 if (s + 1 in call_tree_am.index) else s
        cont_val = np.exp(-r * dt) * 0.5 * (call_tree_am.loc[s, t_next] + call_tree_am.loc[s_up, t_next])
        exercise_val = max(val_clean.loc[s, t] - strike, 0)
        call_tree_am.loc[s, t] = max(cont_val, exercise_val)

# Callable bond values (using both dirty and clean bond trees) for t <= T_o
callable_bond_dirty_am = val_dirty[call_cols] - call_tree_am
callable_bond_clean_am = val_clean[call_cols] - call_tree_am

In [109]:
call_tree_am

,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,4.60526,3.341665,2.315189,1.507514,0.884822,0.446912,0.177591,0.042461,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,5.966348,4.440776,3.174638,2.166475,1.345477,0.730739,0.319550,0.086867,0.000000,0.000000,0.000000,0.000000
2,NaN,NaN,7.609029,5.796589,4.249921,3.035145,1.995240,1.163998,0.563553,0.177161,0.000000,0.000000,0.000000
3,NaN,NaN,NaN,8.671701,7.450967,5.544759,4.138429,2.873827,1.796362,0.967019,0.360534,0.000000,0.000000
4,NaN,NaN,NaN,NaN,10.034062,8.143463,7.043995,5.217558,4.012321,2.667529,1.599512,0.732241,0.000000
5,NaN,NaN,NaN,NaN,NaN,10.208653,9.352437,7.446728,6.508629,4.790580,3.790593,2.504861,1.484465
6,NaN,NaN,NaN,NaN,NaN,NaN,11.175060,9.208351,8.479758,6.642627,5.866449,4.366654,3.577880
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.592844,10.027926,8.101384,7.499592,5.835675,5.226070
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.238893,9.245014,8.778841,6.989063,6.518000


### 3.2.

In which nodes will the American-style callable bond be exercised?

In [110]:
callable_bond_dirty_am

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,97.264141,97.678942,97.722877,95.167257,94.527593,91.357435,90.383523,87.166598,86.276634,83.234494,82.683237,80.182270,80.271327
1,NaN,98.908403,99.758220,98.004514,98.096494,95.661017,95.296872,92.581608,92.047869,89.229230,88.757972,86.297496,86.364216
2,NaN,NaN,100.000000,99.071840,99.984593,98.232033,98.515351,96.413718,96.394988,94.000375,93.782819,91.361871,91.411396
3,NaN,NaN,NaN,99.190007,100.000000,99.169184,100.000000,98.492342,99.076233,97.233284,97.514257,95.490134,95.526543
4,NaN,NaN,NaN,NaN,100.000000,99.332133,100.000000,99.184159,100.000000,98.764611,99.567358,98.081971,98.840717
5,NaN,NaN,NaN,NaN,NaN,99.463209,100.000000,99.366645,100.000000,99.214224,100.000000,98.960505,100.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,99.508414,100.000000,99.395585,100.000000,99.197611,100.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.618510,100.000000,99.535185,100.000000,99.380803,100.000000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,99.642600,100.000000,99.522272,100.000000


In [111]:
callable_bond_clean_am

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,97.264141,96.576442,97.722877,94.064757,94.527593,90.254935,90.383523,86.064098,86.276634,82.131994,82.683237,79.079770,80.271327
1,NaN,97.805903,99.758220,96.902014,98.096494,94.558517,95.296872,91.479108,92.047869,88.126730,88.757972,85.194996,86.364216
2,NaN,NaN,100.000000,97.969340,99.984593,97.129533,98.515351,95.311218,96.394988,92.897875,93.782819,90.259371,91.411396
3,NaN,NaN,NaN,98.087507,100.000000,98.066684,100.000000,97.389842,99.076233,96.130784,97.514257,94.387634,95.526543
4,NaN,NaN,NaN,NaN,100.000000,98.229633,100.000000,98.081659,100.000000,97.662111,99.567358,96.979471,98.840717
5,NaN,NaN,NaN,NaN,NaN,98.360709,100.000000,98.264145,100.000000,98.111724,100.000000,97.858005,100.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,98.405914,100.000000,98.293085,100.000000,98.095111,100.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.516010,100.000000,98.432685,100.000000,98.278303,100.000000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,98.540100,100.000000,98.419772,100.000000


### 3.2.

In which nodes will the American-style callable bond be exercised?

In [112]:
exercise_nodes = []
for t in call_cols:
    for s in call_tree_am.index:
        immediate_exercise_val = max(val_clean.loc[s, t] - strike, 0)
        # If the call value equals the immediate-exercise value (and it's positive),
        # that means early exercise is optimal at this node
        if abs(call_tree_am.loc[s, t] - immediate_exercise_val) < 1e-12 and immediate_exercise_val > 0:
            exercise_nodes.append((t, s))

exercise_nodes

[(0.5, 2),
 (1.0, 3),
 (1.0, 4),
 (1.5, 3),
 (1.5, 4),
 (1.5, 5),
 (1.5, 6),
 (2.0, 4),
 (2.0, 5),
 (2.0, 6),
 (2.0, 7),
 (2.0, 8),
 (2.5, 5),
 (2.5, 6),
 (2.5, 7),
 (2.5, 8),
 (2.5, 9),
 (2.5, 10),
 (3.0, 5),
 (3.0, 6),
 (3.0, 7),
 (3.0, 8),
 (3.0, 9),
 (3.0, 10),
 (3.0, 11),
 (3.0, 12)]

***

# 4. Pricing the Callable - Bermudan

#### This Section is NOT REQUIRED and NOT EXPECTED
Still, it is not much additional work, and some of you may find it interesting. It also illustrates the power of binomial trees in how easily they handle the Bermudan style. 

### 4.1.

Re-do part `3`, but this time with **Bermudan** style exercise. 
* This corresponds to the Freddie Mac bond in `HW 1`.
* Note that the option value tree will now go all the way to $T$.

As a reminder, the Bermudan style can be exercised as early as $T_o$ all the way to $T$. It can only be exercised on specific dates at 3-month intervals, but in our quarterly-spaced tree, this means every node from $T_o$ onward.

### 4.2.

Compare the valuation to the market quote in `HW 1`.

***